In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2020-us-fatality-crss/person.csv
/kaggle/input/2020-us-fatality-crss/vehicle.csv
/kaggle/input/2020-us-fatality-crss/accident.csv
/kaggle/input/2021-us-fatality-crss/person.csv
/kaggle/input/2021-us-fatality-crss/vehicle.csv
/kaggle/input/2021-us-fatality-crss/accident.csv


In [2]:
# Load data
df_a_2021 = pd.read_csv('/kaggle/input/2021-us-fatality-crss/accident.csv')
df_p_2021 = pd.read_csv('/kaggle/input/2021-us-fatality-crss/person.csv')

## Calculate the Weighted_frequency accident_2021

In [3]:
frequency = df_a_2021['MAXSEV_IM'].value_counts().rename('Frequency')
weighted_frequency = df_a_2021.groupby('MAXSEV_IM')['WEIGHT'].sum().rename('Weighted Frequency')

grouped = df_a_2021.groupby('MAXSEV_IM')['WEIGHT']
std_dev = grouped.apply(lambda x: np.std(x, ddof=1)).rename('Std Dev of Wgt Freq')
std_err = (std_dev / np.sqrt(grouped.count())).rename('Std Err of Percent')

total_weighted_frequency = weighted_frequency.sum()
percent = (weighted_frequency / total_weighted_frequency * 100).rename('Percent')
percent_std_err = (std_err / total_weighted_frequency * 100).rename('Std Err of Percent')

result = pd.concat([frequency, weighted_frequency, std_dev, percent, percent_std_err], axis=1)

result.index = result.index.map(df_a_2021.set_index('MAXSEV_IM')['MAXSEV_IMNAME'].to_dict())

print(result)

                              Frequency  Weighted Frequency  \
MAXSEV_IM                                                     
No Apparent Injury (O)            25322        4.333259e+06   
Possible Injury (C)               12190        9.248229e+05   
Suspected Minor Injury (B)         9523        6.400733e+05   
Suspected Serious Injury (A)       5824        1.529867e+05   
Fatal Injury (K)                   1194        3.950800e+04   
Injured, Severity Unknown           132        9.725342e+03   
No person involved                   13        2.513780e+03   
Died Prior to Crash*                  2        4.704124e+01   

                              Std Dev of Wgt Freq    Percent  \
MAXSEV_IM                                                      
No Apparent Injury (O)                  57.230473  71.002860   
Possible Injury (C)                     53.820814  15.153736   
Suspected Minor Injury (B)              54.100905  10.487957   
Suspected Serious Injury (A)            20.503335

In [4]:
result.to_csv('/kaggle/working/result_weightconsider2021.csv', index=True)

In [5]:
df_a_2021.isnull().sum()

CASENUM          0
PSU              0
PSU_VAR          0
PSUSTRAT         0
REGION           0
                ..
NO_INJ_IMNAME    0
ALCOHOL          0
ALCOHOLNAME      0
ALCHL_IM         0
ALCHL_IMNAME     0
Length: 80, dtype: int64

In [6]:
df_a_2021['WEIGHT'] = pd.to_numeric(df_a_2021['WEIGHT'], errors='coerce')

In [7]:
# Validation
df_a_2021.loc[df_a_2021['MAXSEV_IMNAME'] == 'Fatal Injury (K)', 'WEIGHT'].sum()

39507.999999972395

## Merge

In [8]:
pd.set_option('display.max_rows', None) 

In [9]:
# Merge - Right Join
merged_df = pd.merge(df_a_2021, df_p_2021, on='CASENUM', how='right')

In [10]:
# check None value
merged_df.isnull().sum()

CASENUM                   0
PSU_x                     0
PSU_VAR_x                 0
PSUSTRAT_x                0
REGION_x                  0
REGIONNAME_x              0
URBANICITY_x              0
URBANICITYNAME_x          0
STRATUM_x                 0
STRATUMNAME_x             0
PJ_x                      0
WEIGHT_x                  0
PEDS                      0
PERNOTMVIT                0
VE_TOTAL                  0
VE_FORMS_x                0
PVH_INVL                  0
PERMVIT                   0
MONTH_x                   0
MONTHNAME_x               0
DAY_WEEK                  0
DAY_WEEKNAME              0
WKDY_IM                   0
WKDY_IMNAME               0
YEAR                      0
YEARNAME                  0
HOUR_x                    0
HOURNAME_x                0
HOUR_IM                   0
HOUR_IMNAME               0
MINUTE_x                  0
MINUTENAME_x              0
MINUTE_IM                 0
MINUTE_IMNAME             0
HARM_EV_x                 0
HARM_EVNAME_x       

## Drop duplicates

In [11]:
# Drop duplicate column after joining
columns_to_drop = [col for col in merged_df.columns if col.endswith('_y')]
merged_df.drop(columns=columns_to_drop, inplace=True)

In [12]:
# Create an identify column 
merged_df.insert(0, 'ID', range(1, len(merged_df) + 1))

In [13]:
# check None value
merged_df.isnull().sum()

ID                        0
CASENUM                   0
PSU_x                     0
PSU_VAR_x                 0
PSUSTRAT_x                0
REGION_x                  0
REGIONNAME_x              0
URBANICITY_x              0
URBANICITYNAME_x          0
STRATUM_x                 0
STRATUMNAME_x             0
PJ_x                      0
WEIGHT_x                  0
PEDS                      0
PERNOTMVIT                0
VE_TOTAL                  0
VE_FORMS_x                0
PVH_INVL                  0
PERMVIT                   0
MONTH_x                   0
MONTHNAME_x               0
DAY_WEEK                  0
DAY_WEEKNAME              0
WKDY_IM                   0
WKDY_IMNAME               0
YEAR                      0
YEARNAME                  0
HOUR_x                    0
HOURNAME_x                0
HOUR_IM                   0
HOUR_IMNAME               0
MINUTE_x                  0
MINUTENAME_x              0
MINUTE_IM                 0
MINUTE_IMNAME             0
HARM_EV_x           

In [14]:
# Split values
parts = merged_df['STRATUMNAME_x'].str.split('-')
merged_df['STRATUMNAME_x'] = parts.str[1].str.strip()
merged_df[:5]

,ID,CASENUM,PSU_x,PSU_VAR_x,PSUSTRAT_x,REGION_x,REGIONNAME_x,URBANICITY_x,URBANICITYNAME_x,STRATUM_x,...,EMER_USE,EMER_USENAME,ROLLOVER,ROLLOVERNAME,IMPACT1,IMPACT1NAME,FIRE_EXP,FIRE_EXPNAME,MAK_MOD,MAK_MODNAME
0,1,202102916823,51,51,10,3,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",1,Urban Area,10,...,0.0,Not Applicable,0.0,No Rollover,12.0,12 Clock Point,0.0,No or Not Reported,35039.0,Nissan/Datsun 810/Maxima
1,2,202102918622,20,20,25,4,"West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, ...",2,Rural Area,10,...,0.0,Not Applicable,0.0,No Rollover,6.0,6 Clock Point,0.0,No or Not Reported,2405.0,Jeep / Kaiser-Jeep / Willys- Jeep Liberty
2,3,202102918654,75,75,19,3,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",2,Rural Area,10,...,0.0,Not Applicable,0.0,No Rollover,12.0,12 Clock Point,0.0,No or Not Reported,35043.0,Nissan/Datsun Sentra
3,4,202102918674,40,40,15,3,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",1,Urban Area,9,...,0.0,Not Applicable,0.0,No Rollover,12.0,12 Clock Point,0.0,No or Not Reported,49401.0,Toyota 4-Runner
4,5,202102918674,40,40,15,3,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",1,Urban Area,9,...,0.0,Not Applicable,0.0,No Rollover,3.0,3 Clock Point,0.0,No or Not Reported,63032.0,KIA Rio/Rio5


In [15]:
# check the types of all columns
merged_df.dtypes

ID                     int64
CASENUM                int64
PSU_x                  int64
PSU_VAR_x              int64
PSUSTRAT_x             int64
REGION_x               int64
REGIONNAME_x          object
URBANICITY_x           int64
URBANICITYNAME_x      object
STRATUM_x              int64
STRATUMNAME_x         object
PJ_x                   int64
WEIGHT_x             float64
PEDS                   int64
PERNOTMVIT             int64
VE_TOTAL               int64
VE_FORMS_x             int64
PVH_INVL               int64
PERMVIT                int64
MONTH_x                int64
MONTHNAME_x           object
DAY_WEEK               int64
DAY_WEEKNAME          object
WKDY_IM                int64
WKDY_IMNAME           object
YEAR                   int64
YEARNAME               int64
HOUR_x                 int64
HOURNAME_x            object
HOUR_IM                int64
HOUR_IMNAME           object
MINUTE_x               int64
MINUTENAME_x          object
MINUTE_IM              int64
MINUTE_IMNAME 

In [16]:
# change first column from int 64 to String
merged_df['CASENUM'] = merged_df['CASENUM'].astype(str)

## Calculate the Weighted_frequency accident_2020

In [17]:
# Calculate weight
merged_df['is_duplicated'] = merged_df.duplicated(subset='CASENUM', keep='first')
merged_df['unique weight'] = np.where(merged_df['is_duplicated'], 0, merged_df['WEIGHT_x'])

In [18]:
# Validation
merged_df.loc[merged_df['MAXSEV_IMNAME'] == 'Fatal Injury (K)', 'unique weight'].sum()

39507.999999972395

## Output

In [19]:
merged_df.to_csv('/kaggle/working/US Road Fatality modified new.csv', index=False)

## Note
1. Should create a column for "Date" - Combine Year, Month, Hour, Minute. 
2. Don't delete any columns you're uncertain about, especially if you're not sure whether you'll need them later. It will help you save your time.
3. Validate your data.
4. Always understand the meaning of each column in your data. Know your data first!
5. Next -- Compare the data between 2020 and 2021

In [20]:
# # Drop Columns
# merged_df = merged_df.drop(columns=[
#  'PSU','WRK_ZONENAME','PSU_VAR','PSUSTRAT','PEDS','PERNOTMVIT','VE_TOTAL','VE_FORMS','PVH_INVL',
#  'PERMVIT','WKDY_IM','HOUR_IMNAME','MINUTE_IM','EVENT1_IM','MANCOL_IM','RELJCT1_IM','RELJCT2_IM',
#  'INJSEV_IM','HELM_USE','HELM_USENAME','HELM_MIS','AIR_BAG','EJECT_IM','DRINKING','PERALCH_IM','TYP_INT','REL_ROAD',
#  'LGTCON_IM','WEATHR_IM','SCH_BUS','INT_HWY','MAXSEV_IM','NO_INJ_IMNAME','ALCHL_IM','AGE_IMNAME',
#  'PER_TYP','PERALCH_IMNAME','DRUGS','DRUGSNAME', 'DRINKINGNAME', 'EJECTION', 'EJECTIONNAME', 'EJECT_IM', 'REST_USE', 'REST_USENAME', 'REST_MIS'])

In [21]:
# df_p_2020 = pd.read_csv('/kaggle/input/2020-us-fatality-crss/accident.csv')
# df_a_2020 = pd.read_csv('/kaggle/input/2020-us-fatality-crss/accident.csv')